## HS Codes Scraping By Metal ##
A script to find all relevant HS codes for a given keyword (metal)
The results are written to data/<metal_name> in the form of raw HS codes and trade data
The second cell runs this for manufacturing data. this is stored in data/manufacturing/<product name>
The third cell runs this for assembly data from the unu-to-hs file. This is stored in data/assembly/<product name>

In [6]:
import pandas as pd
import os
import re

base_dir = "../data_baci"
reference_file = os.path.join(base_dir, "product_codes_HS12_V202501.csv")
baci_files = [f for f in os.listdir(base_dir) if f.startswith("BACI_HS12_Y") and f.endswith(".csv")]
os.makedirs(os.path.join(base_dir, "outputs"), exist_ok=True)

'/Users/pranavgunhal/Downloads/e-waste research /BACI_HS12_V202501/data_baci'

#get files
reference_df = pd.read_csv(reference_file)
metal_keywords = ["copper", "gold", "silver", "aluminium", "tin", "tungsten", "nickel", "lithium", "cobalt", "lead", "zinc", "rare earth", "graphite", "antimony", "gallium", "germanium", "indium", "tantalum"]
metal_keywords = ["Earth-metals, rare"]



#process each metal
for metal in metal_keywords:
    print(f"\n=== Processing keyword: {metal} ===")
    pattern = re.compile(rf"\b{re.escape(metal.lower())}\b", re.IGNORECASE)

    matched_rows = []
    for _, row in reference_df.iterrows():
        if pattern.search(str(row["description"]).lower()):
            matched_rows.append({
                "metal": metal,
                "hs_code": int(row["code"]),
                "description": row["description"]
            })

    if not matched_rows:
        print(f"No matches for: {metal}")
        continue

    metal_df = pd.DataFrame(matched_rows).drop_duplicates()
    hs_code_set = set(metal_df["hs_code"])

    # Create output directory
    metal_output_dir = os.path.join(base_dir, "data", metal.lower().replace(" ", "_"))
    os.makedirs(metal_output_dir, exist_ok=True)

    # Save matched HS codes
    metal_df.to_csv(os.path.join(metal_output_dir, "hs_codes.csv"), index=False)
    print(f"Saved {len(metal_df)} HS codes for {metal}")

    # extract from each year's file
    trade_rows = []
    for fname in sorted(baci_files):
        fpath = os.path.join(base_dir, fname)
        try:
            df = pd.read_csv(fpath)
            df_filtered = df[df["k"].isin(hs_code_set)].copy()
            year_match = re.search(r"Y(\d{4})", fname)
            if year_match:
                df_filtered["year"] = int(year_match.group(1))
            if not df_filtered.empty:
                trade_rows.append(df_filtered)
                print(f"Included rows from {fname}")
        except Exception as e:
            print(f"Skipped {fname} due to error: {e}")

    if trade_rows:
        combined_df = pd.concat(trade_rows, ignore_index=True)
        combined_df.to_csv(os.path.join(metal_output_dir, "trade_data.csv"), index=False)
        print(f"Saved trade data for {metal}")
    else:
        print(f"No trade data found for {metal}")



=== Processing keyword: Earth-metals, rare ===
Saved 1 HS codes for Earth-metals, rare
Included rows from BACI_HS12_Y2012_V202501.csv
Included rows from BACI_HS12_Y2013_V202501.csv
Included rows from BACI_HS12_Y2014_V202501.csv
Included rows from BACI_HS12_Y2015_V202501.csv
Included rows from BACI_HS12_Y2016_V202501.csv
Included rows from BACI_HS12_Y2017_V202501.csv
Included rows from BACI_HS12_Y2018_V202501.csv


KeyboardInterrupt: 

In [9]:

manufacturing_keywords = {
    "semiconductors": ["integrated circuits", "electronic ic", "semiconductor"],
    # "pcbs": ["printed circuit", "pcb"],
    # "capacitors": ["fixed capacitor", "capacitor"],
    # "resistors": ["fixed resistor", "resistor"],
    # "power_supplies": ["ac/dc converter", "static converter", "power supply"],
    # "batteries": ["lithium-ion accumulator", "battery", "accumulator"],
    # "connectors": ["switching", "protection device", "electrical apparatus"],
    # "storage_devices": ["storage", "solid-state", "magnetic storage", "optical storage", "hard drive", "solid state drive", "storage unit"],
    # "fans": ["fan", "cooling unit"],
    # "leds_lasers": ["led", "diode", "laser"]
}


for category, keywords in manufacturing_keywords.items():
    print(f"\n=== Processing category: {category} ===")
    
    pattern = re.compile(rf"\b({'|'.join([re.escape(k.lower()) for k in keywords])})\b", re.IGNORECASE)

    matched_rows = []
    for _, row in reference_df.iterrows():
        if pattern.search(str(row["description"]).lower()):
            try:
                hs_code = int(row["code"])
                matched_rows.append({
                    "category": category,
                    "hs_code": hs_code,
                    "description": row["description"]
                })
            except (ValueError, TypeError):
                continue

    if not matched_rows:
        print(f"No matches for: {category}")
        continue

    category_df = pd.DataFrame(matched_rows).drop_duplicates()
    hs_code_set = set(category_df["hs_code"])

    # Create output folder under 'manufacturing'
    category_output_dir = os.path.join(base_dir, "outputs", "manufacturing", category.lower().replace(" ", "_"))
    os.makedirs(category_output_dir, exist_ok=True)

    # Save HS code matches
    category_df.to_csv(os.path.join(category_output_dir, "hs_codes.csv"), index=False)
    print(f"Saved {len(category_df)} HS codes for {category}")

    # Extract BACI trade rows
    trade_rows = []
    for fname in sorted(baci_files):
        fpath = os.path.join(base_dir, fname)
        try:
            df = pd.read_csv(fpath)
            df_filtered = df[df["k"].isin(hs_code_set)].copy()
            year_match = re.search(r"Y(\d{4})", fname)
            if year_match:
                df_filtered["year"] = int(year_match.group(1))
            if not df_filtered.empty:
                trade_rows.append(df_filtered)
                print(f"Included rows from {fname}")
        except Exception as e:
            print(f"Skipped {fname} due to error: {e}")

    if trade_rows:
        combined_df = pd.concat(trade_rows, ignore_index=True)
        combined_df.to_csv(os.path.join(category_output_dir, "trade_data.csv"), index=False)
        print(f"Saved trade data for {category}")
    else:
        print(f"No trade data found for {category}")



=== Processing category: semiconductors ===
Saved 16 HS codes for semiconductors
Included rows from BACI_HS12_Y2012_V202501.csv
Included rows from BACI_HS12_Y2013_V202501.csv
Included rows from BACI_HS12_Y2014_V202501.csv
Included rows from BACI_HS12_Y2015_V202501.csv
Included rows from BACI_HS12_Y2016_V202501.csv
Included rows from BACI_HS12_Y2017_V202501.csv
Included rows from BACI_HS12_Y2018_V202501.csv
Included rows from BACI_HS12_Y2019_V202501.csv
Included rows from BACI_HS12_Y2020_V202501.csv
Included rows from BACI_HS12_Y2021_V202501.csv
Included rows from BACI_HS12_Y2022_V202501.csv
Included rows from BACI_HS12_Y2023_V202501.csv
Saved trade data for semiconductors


In [3]:
import pandas as pd
import os
import re

# === CONFIGURATION ===
base_dir = "../data_baci"
reference_file = os.path.join(base_dir, "product_codes_HS12_V202501.csv")
unu_to_hs_file = "../ref/unu-to-hs.csv"  # path to your mapping file
output_base = os.path.join(base_dir, "data", "assembly")
baci_files = sorted([f for f in os.listdir(base_dir) if f.startswith("BACI_HS12_Y") and f.endswith(".csv")])
os.makedirs(output_base, exist_ok=True)

# === LOAD MAPPINGS ===
reference_df = pd.read_csv(reference_file)
unu_df = pd.read_csv(unu_to_hs_file)
unu_df["HS"] = unu_df["HS"].astype(str)

# === GROUP BY CATEGORY ===
category_groups = unu_df.groupby("UNU Key Description")["HS"].apply(set)

# === PROCESS EACH CATEGORY ===
for category, hs_set in category_groups.items():
    category_slug = category.lower().replace(" ", "_").replace("/", "_")
    category_dir = os.path.join(output_base, category_slug)
    os.makedirs(category_dir, exist_ok=True)

    # Save hs_codes.csv
    matched_rows = reference_df[reference_df["code"].astype(str).isin(hs_set)].copy()
    matched_rows = matched_rows.drop_duplicates(subset="code")
    matched_rows["category"] = category
    matched_rows.to_csv(os.path.join(category_dir, "hs_codes.csv"), index=False)
    print(f"[INFO] Saved {len(matched_rows)} HS codes for {category}")

    # === Extract Trade Data ===
    trade_rows = []
    for fname in baci_files:
        fpath = os.path.join(base_dir, fname)
        try:
            df = pd.read_csv(fpath)
            df["k"] = df["k"].astype(str)
            df_filtered = df[df["k"].isin(hs_set)].copy()
            year_match = re.search(r"Y(\d{4})", fname)
            if year_match:
                df_filtered["year"] = int(year_match.group(1))
            if not df_filtered.empty:
                trade_rows.append(df_filtered)
                print(f"[DATA] Included rows from {fname}")
        except Exception as e:
            print(f"[ERROR] Skipped {fname} due to: {e}")

    # Save combined trade_data.csv
    if trade_rows:
        combined_df = pd.concat(trade_rows, ignore_index=True)
        combined_df.to_csv(os.path.join(category_dir, "trade_data.csv"), index=False)
        print(f"[DONE] Trade data saved for {category}")
    else:
        print(f"[WARN] No trade data found for {category}")


[INFO] Saved 3 HS codes for Air Conditioners (household installed and portable)
[DATA] Included rows from BACI_HS12_Y2012_V202501.csv
[DATA] Included rows from BACI_HS12_Y2013_V202501.csv
[DATA] Included rows from BACI_HS12_Y2014_V202501.csv
[DATA] Included rows from BACI_HS12_Y2015_V202501.csv
[DATA] Included rows from BACI_HS12_Y2016_V202501.csv
[DATA] Included rows from BACI_HS12_Y2017_V202501.csv
[DATA] Included rows from BACI_HS12_Y2018_V202501.csv
[DATA] Included rows from BACI_HS12_Y2019_V202501.csv
[DATA] Included rows from BACI_HS12_Y2020_V202501.csv
[DATA] Included rows from BACI_HS12_Y2021_V202501.csv
[DATA] Included rows from BACI_HS12_Y2022_V202501.csv
[DATA] Included rows from BACI_HS12_Y2023_V202501.csv
[DONE] Trade data saved for Air Conditioners (household installed and portable)
[INFO] Saved 1 HS codes for Cameras (f.i. camcorders, foto & digital still cameras)
[DATA] Included rows from BACI_HS12_Y2012_V202501.csv
[DATA] Included rows from BACI_HS12_Y2013_V202501.csv
